# 消融实验：PINN架构组件贡献分析

本Notebook系统评估PINN架构中各组件的贡献，包括：
1. Residual Connections的影响
2. Attention Mechanism的影响
3. Dynamic Weighting vs Fixed Weighting
4. 不同约束权重设置的影响


In [ ]:
import sys
import importlib
sys.path.append('.')

# 重新加载模块以确保使用最新代码
if 'ablation_study' in sys.modules:
    importlib.reload(sys.modules['ablation_study'])
from ablation_study import *

print("消融实验环境已就绪！")
print(f"Cholesky参数维度验证: 对于3-qubit系统(dim=8), output_dim应该是64, 实际={8*8}")


消融实验环境已就绪！
消融实验环境已就绪！
Cholesky参数维度验证: 对于3-qubit系统(dim=8), output_dim应该是64, 实际=64


In [2]:
# 验证维度计算
n_qubits = 3
dim = 2 ** n_qubits
# 对角元素：dim个，每个1个参数
diag_params = dim
# 非对角元素：dim*(dim-1)/2个，每个2个参数（实部+虚部）
off_diag_params = dim * (dim - 1) // 2 * 2
total_params = diag_params + off_diag_params

print(f"对于{n_qubits}-qubit系统 (dim={dim}):")
print(f"  对角元素参数: {diag_params}")
print(f"  非对角元素参数: {off_diag_params}")
print(f"  总参数数量: {total_params}")
print(f"  dim*dim = {dim*dim}")
print(f"  验证: total_params == dim*dim? {total_params == dim*dim}")

# 测试模型初始化
test_model = AblationPINN(n_qubits=3, input_dim=64, hidden_dims=[256, 128])
print(f"\n模型output_dim: {test_model.output_dim}")
print(f"预期output_dim: {dim*dim}")
print(f"维度匹配: {test_model.output_dim == dim*dim}")


对于3-qubit系统 (dim=8):
  对角元素参数: 8
  非对角元素参数: 56
  总参数数量: 64
  dim*dim = 64
  验证: total_params == dim*dim? True

模型output_dim: 64
预期output_dim: 64
维度匹配: True


## 运行消融实验


In [3]:
# 运行消融实验
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用设备: {device}")

results = run_ablation_study(
    n_qubits=3, 
    n_train=3000, 
    n_val=800, 
    epochs=25, 
    device=device
)


使用设备: cpu

开始消融实验 - 3 qubits

生成数据集...

训练配置: Full Model
------------------------------------------------------------
Epoch 5/25 - Train Loss: 0.004572, Val Loss: 0.004607, Fidelity: 0.7653, CV: 3.32e-33
Epoch 10/25 - Train Loss: 0.002949, Val Loss: 0.004259, Fidelity: 0.7978, CV: 3.76e-33
Epoch 15/25 - Train Loss: 0.002075, Val Loss: 0.004095, Fidelity: 0.8100, CV: 3.90e-33
Epoch 20/25 - Train Loss: 0.001656, Val Loss: 0.004018, Fidelity: 0.8152, CV: 3.73e-33
Epoch 25/25 - Train Loss: 0.001556, Val Loss: 0.004013, Fidelity: 0.8158, CV: 3.61e-33
最佳保真度: 0.8158

训练配置: w/o Residual
------------------------------------------------------------
Epoch 5/25 - Train Loss: 0.004621, Val Loss: 0.004662, Fidelity: 0.7584, CV: 3.29e-33
Epoch 10/25 - Train Loss: 0.003157, Val Loss: 0.004239, Fidelity: 0.7917, CV: 3.58e-33
Epoch 15/25 - Train Loss: 0.002413, Val Loss: 0.004094, Fidelity: 0.8052, CV: 3.70e-33
Epoch 20/25 - Train Loss: 0.002055, Val Loss: 0.004049, Fidelity: 0.8100, CV: 3.85e-33
Epoch 

## 生成可视化图表


In [4]:
# 绘制所有结果
plot_ablation_results(results, save_dir='./ablation_results')



结果已保存到: ./ablation_results

结果摘要:
      Configuration Best Fidelity Final Fidelity Final Constraint Violation
         Full Model        0.8158         0.8158                   3.61e-33
       w/o Residual        0.8115         0.8110                   3.29e-33
      w/o Attention        0.7906         0.7902                   3.56e-33
Fixed Weight (0.15)        0.8140         0.8140                   3.66e-33
Fixed Weight (0.05)        0.8139         0.8139                   3.65e-33
Fixed Weight (0.30)        0.8086         0.8086                   3.90e-33
 Baseline (w/o all)        0.7974         0.7967                   4.16e-33


## 生成论文专用图表


In [5]:
# 生成论文中需要的消融研究结果图
def create_paper_figures(results, save_dir='./ablation_results'):
    """创建论文中使用的图表"""
    
    # 1. 保真度对比柱状图（论文用）
    fig, ax = plt.subplots(figsize=(10, 6))
    configs = [r['config'] for r in results]
    fidelities = [r['best_fidelity'] for r in results]
    
    # 使用更专业的配色
    colors = ['#2E86AB' if 'Full' in c else '#A23B72' if 'Baseline' in c else '#F18F01' 
              for c in configs]
    
    bars = ax.bar(configs, fidelities, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel('Fidelity', fontsize=14, fontweight='bold')
    ax.set_xlabel('Model Configuration', fontsize=14, fontweight='bold')
    ax.set_title('Ablation Study: Component Contribution to Reconstruction Fidelity', 
                 fontsize=16, fontweight='bold')
    ax.set_ylim([min(fidelities) * 0.95, max(fidelities) * 1.02])
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # 添加数值标签
    for bar, fid in zip(bars, fidelities):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{fid:.4f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'{save_dir}/ablation_fidelity_paper.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'{save_dir}/ablation_fidelity_paper.pdf', bbox_inches='tight')
    plt.close()
    
    # 2. 组件贡献分解图
    fig, ax = plt.subplots(figsize=(12, 6))
    
    baseline_fid = next(r['best_fidelity'] for r in results if 'Baseline' in r['config'])
    
    # 计算每个组件的贡献
    components = []
    contributions = []
    
    # Residual贡献
    no_residual = next((r['best_fidelity'] for r in results if 'w/o Residual' in r['config']), baseline_fid)
    residual_contrib = (next(r['best_fidelity'] for r in results if 'Full' in r['config']) - no_residual) / baseline_fid * 100
    components.append('Residual\\nConnections')
    contributions.append(residual_contrib)
    
    # Attention贡献
    no_attention = next((r['best_fidelity'] for r in results if 'w/o Attention' in r['config']), baseline_fid)
    attention_contrib = (next(r['best_fidelity'] for r in results if 'Full' in r['config']) - no_attention) / baseline_fid * 100
    components.append('Attention\\nMechanism')
    contributions.append(attention_contrib)
    
    # Dynamic Weighting贡献
    fixed_weight = next((r['best_fidelity'] for r in results if 'Fixed Weight (0.15)' in r['config']), baseline_fid)
    dynamic_contrib = (next(r['best_fidelity'] for r in results if 'Full' in r['config']) - fixed_weight) / baseline_fid * 100
    components.append('Dynamic\\nWeighting')
    contributions.append(dynamic_contrib)
    
    bars = ax.bar(components, contributions, color=['#2E86AB', '#A23B72', '#F18F01'], 
                 alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel('Improvement over Baseline (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Component', fontsize=14, fontweight='bold')
    ax.set_title('Individual Component Contribution to Performance', 
                 fontsize=16, fontweight='bold')
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # 添加数值标签
    for bar, contrib in zip(bars, contributions):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{contrib:+.2f}%',
                ha='center', va='bottom' if contrib > 0 else 'top', 
                fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{save_dir}/ablation_component_contribution_paper.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'{save_dir}/ablation_component_contribution_paper.pdf', bbox_inches='tight')
    plt.close()
    
    print("论文专用图表已生成！")

create_paper_figures(results)


论文专用图表已生成！


## 详细结果分析


In [ ]:
# 显示结果表格
import pandas as pd
baseline_fid = next(r['best_fidelity'] for r in results if 'Baseline' in r['config'])

df = pd.DataFrame([
    {
        'Configuration': r['config'],
        'Best Fidelity': f"{r['best_fidelity']:.4f}",
        'Final Fidelity': f"{r['final_fidelity']:.4f}",
        'Final CV': f"{r['final_cv']:.2e}",
        'Improvement over Baseline (%)': f"{((r['best_fidelity'] - baseline_fid) / baseline_fid * 100):.2f}"
    }
    for r in results
])
print(df.to_string(index=False))


      Configuration Best Fidelity Final Fidelity Final CV Improvement over Baseline (%)
         Full Model        0.8158         0.8158 3.61e-33                          2.30
       w/o Residual        0.8115         0.8110 3.29e-33                          1.77
      w/o Attention        0.7906         0.7902 3.56e-33                         -0.86
Fixed Weight (0.15)        0.8140         0.8140 3.66e-33                          2.09
Fixed Weight (0.05)        0.8139         0.8139 3.65e-33                          2.07
Fixed Weight (0.30)        0.8086         0.8086 3.90e-33                          1.41
 Baseline (w/o all)        0.7974         0.7967 4.16e-33                          0.00
